# Location Aware Generative Advesarial Network

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torchvision.transforms as transforms
from torchvision.utils import save_image

In [0]:
import nn_local as nn_

### Discriminator

In [0]:
class Discriminator(nn.Module):
    def __init__(self, ngpu=1):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.model = nn.Sequential(
                    # Block I
                    nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2),
                    nn.LeakyReLU(negative_slope=0.3, inplace=True),
                    nn.Dropout(p=0.2, inplace=True),

                    # Block II
                    nn.ZeroPad2d(padding=2),
                    nn_.Conv2dLocal(in_height=29, in_width=29, in_channels=32, out_channels=8, kernel_size=5, stride=2),
                    nn.LeakyReLU(negative_slope=0.3, inplace=True),
                    nn.BatchNorm2d(num_features=8, momentum=0.99, eps=1e-3),
                    nn.Dropout(p=0.2, inplace=True),

                    # Block III
                    nn.ZeroPad2d(padding=2),
                    nn_.Conv2dLocal(in_height=17, in_width=17, in_channels=8, out_channels=8, kernel_size=5, stride=1),
                    nn.LeakyReLU(negative_slope=0.3, inplace=True),
                    nn.BatchNorm2d(num_features=8, momentum=0.99, eps=1e-3),
                    nn.Dropout(p=0.2, inplace=True),

                    # Block IV
                    nn.ZeroPad2d(padding=2),
                    nn_.Conv2dLocal(in_height=17, in_width=17, in_channels=8, out_channels=8, kernel_size=5, stride=2),
                    nn.LeakyReLU(negative_slope=0.3, inplace=True),
                    nn.BatchNorm2d(num_features=8, momentum=0.99, eps=1e-3),
                    nn.Dropout(p=0.2, inplace=True),

                    # Block V
                    nn.AvgPool2d(kernel_size=2),
                    nn.Flatten()
                  )
        
    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output.view(-1, 1).squeeze(1)